In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


page = requests.get(url)

doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')

In [3]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')

col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [5]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [6]:
[len(C) for (title,C) in col]

[287, 287, 287]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [21]:
df.head()

,Postcode,Borough,Neighbourhood
0,Downtown Toronto,Adelaide,M5H
1,Scarborough,Agincourt,M1S
2,Scarborough,Agincourt North,M1V
3,Etobicoke,Albion Gardens,M9V
4,Etobicoke,Alderwood,M8W


In [22]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Postcode,Borough,Neighbourhood
0,M5H,Downtown Toronto,Adelaide
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,Agincourt North
3,M9V,Etobicoke,Albion Gardens
4,M8W,Etobicoke,Alderwood


In [23]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M5H,Downtown Toronto,Adelaide
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,Agincourt North
3,M9V,Etobicoke,Albion Gardens
4,M8W,Etobicoke,Alderwood


In [24]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

df = df.reset_index(drop=True)

df.head(10)

,Postcode,Borough,Neighbourhood
0,M5H,Downtown Toronto,Adelaide
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,Agincourt North
3,M9V,Etobicoke,Albion Gardens
4,M8W,Etobicoke,Alderwood
5,M3H,North York,Bathurst Manor
6,M5V,Downtown Toronto,Bathurst Quay
7,M2K,North York,Bayview Village
8,M9V,Etobicoke,Beaumond Heights
9,M5M,North York,Bedford Park


In [25]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,M1A,Not assigned
1,M1B,Scarborough,"Malvern ,Rouge"
2,M1C,Scarborough,"Highland Creek ,Port Union ,Rouge Hill"
3,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park"
8,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge"
9,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West"


In [26]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [27]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [28]:
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood
160,M9A,Queen's Park,Queen's Park


In [29]:
df.shape

(180, 3)

In [30]:
df.to_csv(r'df_can.csv')